with current system (i.e. a for every gene-org combo) can pull up all results for a single gene with
```
cat *myosin_VIIA_summary.csv
```
can then pass it to
```
|cut -d "," -f 1 | cut -d ":" -f 2  > entrez_test_mult_queries.txt
```
this should be a full list of everything for that gene,
can name the file better later


This gives us more than we had, and it seems like it worked fine

The Question is, do we want to make a better way of parsing the results?

yes, probably

THinking about modifying our current function of parsing loop to create the file name and open it automatically
but how to not overwrite it?

when you open the file
use "a", this will append to the file
not "w", this will overwrite the file as it is written to repeatedly


# Altering the parse and summarize file to separate out the sequences by gene

In [1]:
def parse_and_summarize(blast_output_xml):
    #  goes through the output of a BLAST xml file and finds the relevant stats to summarize the search
    from Bio.Blast import NCBIXML
    from Bio.SeqRecord import SeqRecord
    #import the required libraries
    
    for file_name in blast_output_xml:
        result_handle = open(str(file_name), "r") #sets the result handle
        blast_records = NCBIXML.parse(result_handle)
        #need to use parse if it has multiple records in it
        for blast_record in blast_records:
            org_desig=file_name.split("_")[2].split("[")[0].replace(" ", "_")
            #properly formats the taxa id so we can use it to name things
            
            homo_sapiens = "[Homo sapiens]"
            blast_query=blast_record.query
            if homo_sapiens in blast_query:
                gene_name=blast_record.query.split("|")[4].split("[")[0].replace(" ", "_").replace("_protein", "").replace("_isoform_b3", "")
                formated_gene_name = gene_name[1:-1]
            else:
                formated_gene_name=blast_query.split("|")[4].split("_")[0]
            #this conditional properly formats the gene name so we can use it to name things
                #Basically there are 2 formats of sequence names I used to do the search
                #this statement switches the naming convention depending on which is used
                #it's not very general, so in the future I will make sure to properly name my search sequences
                #that should make this statement unnecessary 
            
            
            output_file_name=(formated_gene_name + "_summary.csv")    
            output_file=open(output_file_name, "a")
            #this puts together a nicely named file for each gene and species
            
            alignment_file_contents=""
            #setting up an empty list to hold the output of the following loop
            for alignment in blast_record.alignments:

                score_counter=[]
                e_val_counter=[]

                for hsp in alignment.hsps:
                    score=hsp.score
                    score_counter.append(score)
                    evalue=hsp.expect
                    e_val_counter.append(evalue)
                
                seq_designation = alignment.title.split("|")
                gi_number = seq_designation[1]
                ref_number = seq_designation[3]
                annotation = seq_designation[4]
                #breaking apart the annoation of the sequence name
                #otherwise we'd have way too many delimiters within delimiters
                output_line = str("gi:" + gi_number + ","+
                                 "ref_number:" + ref_number + "," +
                                 "annotation:" + annotation+ "," +
                                "best_score:"+ str(max(score_counter)) + ","+
                                "evalue:" + str(min(e_val_counter)) + ","
                                "length:" + str(alignment.length)+"\n")
                #this sets up the output for each alignment
                
                alignment_file_contents = alignment_file_contents + output_line
                #this writes what we came up with to the holder string
            
            output_file.write(alignment_file_contents)
            #writing the output to the alignment file
            output_file.close()

        result_handle.close()
        

In [2]:
list_of_blast_xml = ["USH_Search_txid45351[ORGN].xml", "USH_Search_txid400682[ORGN].xml"]

In [3]:
parse_and_summarize(list_of_blast_xml)


# Well that was much less of a problem than I thought it would be

### Only real problem comes with the issue of needing to delete these guys when I want to rerun it with a larger data set.
ultimately pretty minor

#Let's use bash to get us a list of all the Blast docs!

I think a few of the BLAST results may have gone missing?
bash says we have 59
from list of orgs to search should have 61?



#Let's find out! 

#Can do this later I say werid formatting things will come up and I will get tired


full_taxa_file_name=open("/home/eeb177-student/Desktop/eeb-177/project/sandbox/Testrun_multi_genes_same_org/full_list_taxa_NCBI.txt", "r")
full_taxa_list= full_taxa_file_name.readlines()
full_taxa_list

#basically want to create a logical comparison for elements in one vector within other vector

Let's just run it for the taxa we can extract from Bash easily

In [9]:
file_List_of_Blast_XMLs = open("/home/eeb177-student/Desktop/eeb-177/project/sandbox/Testrun_multi_genes_same_org/Batch_run_workspace/list_of_BLAST_XML.txt", "r")
list_of_BLAST_XMLs = file_List_of_Blast_XMLs.read().splitlines()

In [12]:
list_of_BLAST_XMLs

['USH_Search_txid10090[ORGN].xml',
 'USH_Search_txid10224[ORGN].xml',
 'USH_Search_txid10228[ORGN].xml',
 'USH_Search_txid109871[ORGN].xml',
 'USH_Search_txid1202447[ORGN].xml',
 'USH_Search_txid134558[ORGN].xml',
 'USH_Search_txid164328[ORGN].xml',
 'USH_Search_txid192875[ORGN].xml',
 'USH_Search_txid203908[ORGN].xml',
 'USH_Search_txid225164[ORGN].xml',
 'USH_Search_txid227086[ORGN].xml',
 'USH_Search_txid2769[ORGN].xml',
 'USH_Search_txid281847[ORGN].xml',
 'USH_Search_txid2850[ORGN].xml',
 'USH_Search_txid28583[ORGN].xml',
 'USH_Search_txid3067[ORGN].xml',
 'USH_Search_txid34488[ORGN].xml',
 'USH_Search_txid361139[ORGN].xml',
 'USH_Search_txid3702[ORGN].xml',
 'USH_Search_txid37621[ORGN].xml',
 'USH_Search_txid37653[ORGN].xml',
 'USH_Search_txid400682[ORGN].xml',
 'USH_Search_txid44689[ORGN].xml',
 'USH_Search_txid45351[ORGN].xml',
 'USH_Search_txid46433[ORGN].xml',
 'USH_Search_txid4837[ORGN].xml',
 'USH_Search_txid4932[ORGN].xml',
 'USH_Search_txid5011[ORGN].xml',
 'USH_Search_tx

In [13]:
len(list_of_BLAST_XMLs)

59

In [14]:
parse_and_summarize(list_of_BLAST_XMLs)